In [1]:
import numpy as np
import pandas as pd
import os

from pathlib import Path
import getpass
import os
import subprocess
from typing import Dict

In [88]:
vars = ['PIS','identificad', 'municipio',
        'remmedr','remdezr', 'horascontr',
        'dtadmissao','causadesli','mesdesli', 'empem3112',
        'clascnae20', 'natjuridica',
        'ocup2002','genero','tempempr','raca_cor','grinstrucao', ]
       
        

        
        

In [135]:

df_total = pd.read_stata("/Users/luisg/Desktop/SteppingStones_Parent/RAIS/output/data/full/RAIS_2008.dta", convert_categoricals=False)

df = pd.read_stata("/Users/luisg/Desktop/SteppingStones_Parent/RAIS/output/data/full/RAIS_2008.dta", columns=vars, convert_categoricals=False)



In [136]:
df_total.columns

Index(['PIS', 'CPF', 'numectps', 'nome', 'identificad', 'radiccnpj',
       'municipio', 'tpvinculo', 'empem3112', 'tipoadm', 'dtadmissao',
       'causadesli', 'diadesli', 'mesdesli', 'ocupacao94', 'ocup2002',
       'grinstrucao', 'genero', 'dtnascimento', 'nacionalidad', 'portdefic',
       'tpdefic', 'raca_cor', 'remdezembro', 'remdezr', 'remmedia', 'remmedr',
       'tempempr', 'tiposal', 'salcontr', 'ultrem', 'horascontr', 'clascnae95',
       'clascnae20', 'sbclas20', 'tamestab', 'natjuridica', 'tipoestbl',
       'indceivinc', 'ceivinc', 'indalvara', 'indpat', 'indsimples',
       'qtdiasafas', 'causafast1', 'causafast2', 'causafast3', 'diainiaf1',
       'diainiaf2', 'diainiaf3', 'diafimaf1', 'diafimaf2', 'diafimaf3',
       'mesiniaf1', 'mesiniaf2', 'mesiniaf3', 'mesfimaf1', 'mesfimaf2',
       'mesfimaf3'],
      dtype='object')

In [137]:
causadesliUnicos = df.causadesli.unique().tolist()


causadesliUnicos

[0,
 11,
 12,
 21,
 10,
 31,
 30,
 60,
 20,
 40,
 50,
 71,
 70,
 73,
 76,
 75,
 74,
 72,
 62,
 22,
 79,
 78,
 80,
 63,
 64]

In [138]:
empem3112Unicos = df.empem3112.unique().tolist()
empem3112Unicos

[1, 0]

In [142]:
df['dtadmissao'].head()
df['dtadmissao'].dtype

df['dtadmissao_date'] = pd.to_datetime(df['dtadmissao'], format="%d%m%Y", errors='coerce')
df[['dtadmissao','dtadmissao_date']].head()

,dtadmissao,dtadmissao_date
0,28072008,2008-07-28
1,14031986,1986-03-14
2,14062004,2004-06-14
3,11052000,2000-05-11
4,13022008,2008-02-13


In [145]:
df['emp_in_dec'] = ((df['empem3112']==1 )& (df['tempempr']>1)).astype(int)
df[['empem3112','tempempr','emp_in_dec']].head()

,empem3112,tempempr,emp_in_dec
0,1,5.0,1
1,1,273.5,1
2,0,49.3,0
3,1,103.6,1
4,0,1.9,0


In [146]:
# what is the average mean earnings across emp in dec and non emp in dec? 
df.groupby('emp_in_dec')[['remmedr', 'remdezr']].agg(['count', 'mean','min','max'])

# best to select spells by hourly remmedr.

remmedr                             remdezr                    \
             count         mean  min       max   count         mean  min   
emp_in_dec                                                                 
0           213531   982.872771  0.0  60969.69  213531   197.289881  0.0   
1           383533  1374.157816  0.0  59562.85  383533  1455.295140  0.0   

                      
                 max  
emp_in_dec            
0           60576.00  
1           62146.06

In [147]:
# define hourly wages:
df['remmedr_h'] = (df['remmedr'])/(df['horascontr']*4.348)
df['remdezr_h'] = (df['remdezr'])/(df['horascontr']*4.348) 

df[['PIS','remmedr','horascontr','remmedr_h','remdezr','remdezr_h']].head()

,PIS,remmedr,horascontr,remmedr_h,remdezr,remdezr_h
0,12749329932,4026.57,44,21.047138,1485.66,7.765639
1,10701706985,14784.73,36,94.454219,15877.50,101.435526
2,12501500581,1541.59,36,9.848653,0.00,0.000000
3,12749437603,975.52,30,7.478688,1991.58,15.268169
4,17015208715,528.91,44,2.764646,0.00,0.000000


In [ ]:
# select only one spell according to criteria:

# 1) highest horascontr:


In [148]:
worker_id  = df['PIS']
hours = df['horascontr'].fillna(-np.inf)
avg_w_h = df['remmedr_h'].fillna(-np.inf)

In [149]:
max_hours = hours.groupby(worker_id).transform('max')
max_hours.head()

rank1 = hours.eq(max_hours)

pd.concat([hours.head(10), max_hours.head(10), rank1.head(10)], axis=1)

,horascontr,horascontr,horascontr
0,44,44,True
1,36,36,True
2,36,36,True
3,30,30,True
4,44,44,True
5,44,44,True
6,44,44,True
7,44,44,True
8,44,44,True
9,44,44,True


In [150]:
max_avg_w_and_rank1 = avg_w_h.where(rank1).groupby(worker_id).transform('max')
rank2 = rank1 & avg_w_h.eq(max_avg_w_and_rank1)
pd.concat([worker_id.head(10), hours.head(10), max_hours.head(10), rank1.head(10), avg_w_h.head(10), max_avg_w_and_rank1.head(10), rank2.head(10)], axis=1)

,PIS,horascontr,horascontr,horascontr,remmedr_h,remmedr_h,0
0,12749329932,44,44,True,21.047138,21.047138,True
1,10701706985,36,36,True,94.454219,94.454219,True
2,12501500581,36,36,True,9.848653,9.848653,True
3,12749437603,30,30,True,7.478688,7.478688,True
4,17015208715,44,44,True,2.764646,2.764646,True
5,13186188546,44,44,True,3.138799,3.138799,True
6,12441769881,44,44,True,2.796688,2.796688,True
7,12622006448,44,44,True,2.299908,2.299908,True
8,10260715163,44,44,True,33.794378,33.794378,True
9,12789060535,44,44,True,3.880572,3.880572,True


In [151]:
rng = np.random.default_rng(123)
df['_rand'] = rng.random(len(df))
rand = df['_rand']


In [152]:
idx = (
    df.loc[rank2]
      .groupby(worker_id[rank2])['_rand']   # <- use the column name here
      .idxmax()
)

# Grab the winning rows
df_one_spell = df.loc[idx].copy()
df_one_spell.head()

,PIS,identificad,municipio,remmedr,remdezr,horascontr,dtadmissao,causadesli,mesdesli,empem3112,...,ocup2002,genero,tempempr,raca_cor,grinstrucao,dtadmissao_date,emp_in_dec,remmedr_h,remdezr_h,_rand
65762,10000004283,06226583000150,210980,556.70,1358.30,40,01042008,12,12,0,...,331205,0,8.9,-1,7,2008-04-01,0,3.200897,7.809913,0.659071
53743,10000074028,60448040000122,355030,4905.16,4866.05,20,22111978,0,0,1,...,223115,1,361.2,9,9,1978-11-22,1,56.407084,55.957337,0.452747
14830,10000086212,00720011000146,530010,1869.32,0.00,14,07082006,11,7,0,...,234515,1,23.7,6,9,2006-08-07,0,30.709029,0.000000,0.310429
20787,10000121182,92816560000307,420540,8769.73,8302.86,30,01032007,0,0,1,...,252210,1,21.9,2,9,2007-03-01,1,67.231907,63.652714,0.915486
81666,10000146576,01440615000100,314330,600.00,600.00,5,01102008,0,0,1,...,234505,1,2.9,2,9,2008-10-01,1,27.598896,27.598896,0.475262


In [153]:
df_one_spell['PIS'].is_unique

True

In [154]:
df['municipio'].dtypes

dtype('int32')

In [155]:
df['causadesli'].value_counts()


causadesli
0     394753
11    100567
21     42615
12     40672
31     12007
10      2600
30      1260
60       602
71       548
20       413
70       388
76       132
40       130
50        78
72        66
79        43
74        40
75        37
80        30
78        26
73        20
62        18
22        12
63         5
64         2
Name: count, dtype: int64